# Trabajo Práctico Nro. 2
## Mineria de Datos 2024

### Integrantes:
 * Pace, Bruno. Legajo: P-5295/7.
 * Sancho Almenar, Mariano. Legajo: S-5778/8.

In [1]:
#Manejo de datos
import pandas as pd
import numpy as np

#Gráficos
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

"""
#Modelado
!pip install kneed
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE, Isomap
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from kneed import KneeLocator
import seaborn as sns

"""

#Manejo de warnings
import warnings
warnings.filterwarnings('ignore')

# Lectura df Compañias

In [2]:
df_companies: pd.DataFrame = pd.read_csv('../data/1000_Companies.csv')
df_pokemon: pd.DataFrame = pd.read_csv('../data/PokemonDBCart.csv')

## Exploración df companies

In [3]:
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RyD_Spend       1000 non-null   float64
 1   Administration  1000 non-null   float64
 2   MarketingSpend  1000 non-null   float64
 3   State           1000 non-null   object 
 4   Profit          1000 non-null   float64
dtypes: float64(4), object(1)
memory usage: 39.2+ KB


In [4]:
df_companies.sample(10)

,RyD_Spend,Administration,MarketingSpend,State,Profit
478,21362.0,113611.9331,109997.0765,New York,67282.27141
345,60170.0,119343.2753,184877.2185,California,100435.61090
967,29723.0,114846.7236,126129.6488,California,74425.00156
268,116953.0,127729.2474,294440.1721,New York,148944.83780
184,29050.0,114747.3319,124831.0935,California,73850.06347
618,159000.0,133938.9399,375569.9734,Florida,184865.22710
525,153919.0,133188.5546,365766.1700,New York,180524.57270
105,44699.0,117058.4475,155025.8817,New York,87218.86913
634,127074.0,129223.9628,313968.6692,California,157591.12080
143,156016.0,133498.2492,369812.3372,New York,182316.02170


Queremos saber si en la columna RyD_Spend hay decimales que no sean 0.

In [5]:
def check_decimals(x: float) -> bool:
    """
    Función auxiliar para determinar si un numero tiene o no parte decimal mayor a 0.
    """
    decimals = x - int(x)
    return decimals != 0

In [6]:
df_companies['decimals_ryd_spend'] = df_companies['RyD_Spend'].apply(check_decimals)

In [7]:
df_companies['decimals_ryd_spend'].value_counts()

decimals_ryd_spend
False    949
True      51
Name: count, dtype: int64

Comprobamos lo que queríamos, por ende ya esta columna no es necesaria. Decidimos dropearla.

In [8]:
df_companies = df_companies.drop('decimals_ryd_spend', axis=1)

Chequeamos que las columnas que quedaron son las originales.

In [9]:
df_companies.columns

Index(['RyD_Spend', 'Administration', 'MarketingSpend', 'State', 'Profit'], dtype='object')

Efectivamente sí hay decimales en esa columna.

In [10]:
df_companies.describe()

,RyD_Spend,Administration,MarketingSpend,Profit
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,81668.927200,122963.897612,226205.058419,119546.164656
std,46537.567891,12613.927535,91578.393542,42888.633848
min,0.000000,51283.140000,0.000000,14681.400000
25%,43084.500000,116640.684850,150969.584600,85943.198543
50%,79936.000000,122421.612150,224517.887350,117641.466300
75%,124565.500000,129139.118000,308189.808525,155577.107425
max,165349.200000,321652.140000,471784.100000,476485.430000


Vemos que hay demasiados decimales que no aportan información extra. Por simplicidad y prolijidad, redondeamos a dos decimales las columnas numericas de tipo float.

In [11]:
df_companies[['RyD_Spend','Administration','MarketingSpend', 'Profit']] = df_companies[
    ['RyD_Spend','Administration','MarketingSpend', 'Profit']].apply(lambda x: round(x,2))

### Visualizaciones

In [15]:
px.box(df_companies[
    ['RyD_Spend','Administration','MarketingSpend', 'Profit']], 
    title='Boxplot de las variables de compañias')

Observamos que hay potenciales outliers en las columnas Administration y Profit. Más adelante lo chequeamos.

In [29]:
px.histogram(
    df_companies[['RyD_Spend','Administration','MarketingSpend', 'Profit']],
    title = 'Distribuciones de las variables'
)

- Administration tiene una distribución normal

In [31]:
df_companies[['Administration', 'Profit']].describe()

,Administration,Profit
count,1000.000000,1000.000000
mean,122963.897650,119546.164700
std,12613.927637,42888.633878
min,51283.140000,14681.400000
25%,116640.685000,85943.200000
50%,122421.610000,117641.465000
75%,129139.117500,155577.107500
max,321652.140000,476485.430000


In [38]:
print(f'El cuantil 0.99 de Administration es: {round(df_companies['Administration'].quantile(.99),2)}')
print(f'El cuantil 0.01 de Administration: {round(df_companies['Administration'].quantile(.01))}')
print(f'El cuantil 0.99 de Profit es: {round(df_companies['Profit'].quantile(.99),2)}')

El cuantil 0.99 de Administration es: 152705.22
El cuantil 0.01 de Administration: 99809
El cuantil 0.99 de Profit es: 185033.03


In [ ]:
corr_matrix = df_companies[
    ['RyD_Spend','Administration','MarketingSpend', 'Profit']].corr()
px.imshow(corr_matrix,
          text_auto=True,
          aspect='auto',
          zmin=-1,
          zmax=1)

Vemos correlaciones fuertes:
 - Profit - MarketingSpend.
 - Profit - RyD_Spend.
 - MarketingSpend - RyDSpend.

 Hay una correlación que no es particularmente fuerte pero que es de destacar:
 - Profit - Administration.

 Analizando esto con el contexto que tenemos de las ganancias de las empresas con respecto a los gastos de Marketing y Desarrollo, no es descabellado pensar que mientras más se inviertan en estas áreas, más reditos económicos podemos conseguir. 
 Vemos una tendencia que a medida que se gasta más en personal de administración, más se gana.